In [1]:
# ! pip install pyrtools
# ! pip install opencv-python

In [2]:
import cv2
import numpy as np
import pyrtools as pt
from scipy import signal
from PIL import Image

## COLOR CLUTTER

In [3]:
def conv2(x, y, mode='same'):
    return np.rot90(signal.convolve2d(np.rot90(x, 2), np.rot90(y, 2), mode=mode), 2)


def RRoverlapconv(kernel, in_):
    out = conv2(in_, kernel, mode='same')
    
    rect = np.ones_like(in_)

    overlapsum = conv2(rect, kernel, 'same')
    out = np.sum(kernel) * out / overlapsum
    return out



def RRgaussfilter1D(halfsupport, sigma, center=0):
    t = list(range(-halfsupport, halfsupport+1))
    kernel = [np.exp(-(x-center) **2 /(2* sigma ** 2)) for x in t]
    kernel = kernel/sum(kernel)
    
    return kernel
    

def computeColorClutter(L_pyr, a_pyr, b_pyr, sigD):
    
    numlevels = len(L_pyr);

    
    if len(a_pyr)!=numlevels or len(b_pyr)!=numlevels:
        print('L, a, and b channels must have the same number of levels in the Gaussian pyramid\n')

    covMx = {}
    clutter_levels = [0] * numlevels
    DL = [0] * numlevels
    Da = [0] * numlevels
    Db = [0] * numlevels
    
    
    deltaL2 = 0.0007 ** 2
    deltaa2 = 0.1 ** 2
    deltab2 = 0.05 ** 2
    
    bigG = RRgaussfilter1D(round(2*sigD), sigD)[:,None]
    
    for i in range(0, numlevels):
        DL[i] = RRoverlapconv(bigG, L_pyr[(i,0)])
        DL[i] = RRoverlapconv(bigG.T, DL[i])   # E(L)
        Da[i] = RRoverlapconv(bigG, a_pyr[(i,0)])
        Da[i] = RRoverlapconv(bigG.T, Da[i])   # E(a)
        Db[i] = RRoverlapconv(bigG, b_pyr[(i,0)]);
        Db[i] = RRoverlapconv(bigG.T, Db[i])    # E(b)

    
        # dict idea
        covMx[(i,0,0)] = RRoverlapconv(bigG, L_pyr[(i,0)] ** 2)
        covMx[(i,0,0)] = RRoverlapconv(bigG.T, covMx[(i,0,0)]) - DL[i] ** 2 + deltaL2  # cov(L,L) + deltaL2
        covMx[(i,0,1)] = RRoverlapconv(bigG, L_pyr[(i,0)] * a_pyr[(i,0)])
        covMx[(i,0,1)] = RRoverlapconv(bigG.T, covMx[(i,0,1)]) - DL[i] * Da[i]        # cov(L,a)
        covMx[(i,0,2)] = RRoverlapconv(bigG, L_pyr[(i,0)] * b_pyr[(i,0)])
        covMx[(i,0,2)] = RRoverlapconv(bigG.T, covMx[(i,0,2)]) - DL[i] * Db[i]        # cov(L,b)
        covMx[(i,1,1)] = RRoverlapconv(bigG, a_pyr[(i,0)] ** 2)
        covMx[(i,1,1)] = RRoverlapconv(bigG.T, covMx[(i,1,1)]) - Da[i] ** 2 + deltaa2  # cov(a,a) + deltaa2
        covMx[(i,1,2)] = RRoverlapconv(bigG, a_pyr[(i,0)] * b_pyr[(i,0)])
        covMx[(i,1,2)] = RRoverlapconv(bigG.T, covMx[(i,1,2)]) - Da[i] * Db[i]        # cov(a,b)
        covMx[(i,2,2)] = RRoverlapconv(bigG, b_pyr[(i,0)] ** 2)    
        covMx[(i,2,2)] = RRoverlapconv(bigG.T, covMx[(i,2,2)]) - Db[i] ** 2 + deltab2;  # cov(b,b) + deltab2

    

        
        detIm = covMx[(i,0,0)]*(covMx[(i,1,1)]*covMx[(i,2,2)]-covMx[(i,1,2)]*covMx[(i,1,2)])\
        - covMx[(i,0,1)]*(covMx[(i,0,1)]*covMx[(i,2,2)]-covMx[(i,1,2)]*covMx[(i,0,2)])\
        + covMx[(i,0,2)]*(covMx[(i,0,1)]*covMx[(i,1,2)]-covMx[(i,1,1)]*covMx[(i,0,2)])

        
        clutter_levels[i] = np.sqrt(detIm) ** (1/3)
    return clutter_levels

In [4]:
def colorClutter(inputImage, numlevels, pool_sigma=3, pix=1):
    
    if isinstance(inputImage, list):
        L_pyr = inputImage[0]
        a_pyr = inputImage[1]
        b_pyr = inputImage[2]
        
    else:
        pass #TODO
        return 0
    
    clutter_levels = computeColorClutter(L_pyr, a_pyr, b_pyr, pool_sigma);
    
    kernel_1d = np.array([[0.05, 0.25, 0.4, 0.25, 0.05]])
    kernel_2d = signal.convolve2d(kernel_1d, kernel_1d.T)
    
    clutter_map = clutter_levels[0]
    for scale in range(1,numlevels):
        clutter_here = clutter_levels[scale]
        
        for kk in range(scale, 2, -1):
            # TODO
#             start= np.array([[0,0]])
#             step = np.array([[2,2]])
#             stop = step * (np.floor( (start - np.ones_like(start))/step) + len(clutter_here))
            clutter_here = pt.upConv(image=clutter_here, filt=kernel_2d, edge_type='reflect1', step=[2,2], start=[0,0])
    
        common_sz = min(clutter_map.shape[0], clutter_here.shape[0]), min(clutter_map.shape[1], clutter_here.shape[1])
        for i in range(0, common_sz[0]):
            for j in range(0, common_sz[1]):
                 clutter_map[i][j] = max(clutter_map[i][j], clutter_here[i][j])

    
    return clutter_levels, clutter_map

## CONTRAST CLUTTER

In [5]:
# Test is passed: Diffrent test cases manually checked by MATLAB
def DoG1filter(a,sigma):
    
    sigi = 0.71 * sigma
    sigo = 1.14 * sigma
    
    t = range(-a, a+1)
    
    gi = [np.exp(-x ** 2 /(2 * sigi ** 2)) for x in t]
    gi = gi/sum(gi)
    go = [np.exp(- x ** 2/(2 * sigo ** 2)) for x in t]
    go = go/sum(go)
    return gi,go

In [6]:
# Test is passed: Diffrent test cases manually checked by MATLAB
def addborder(im,xbdr,ybdr,arg):
    """
    imnew = addborder(im,xborder,yborder,arg)  Make image w/added border.
    imnew = addborder(im,5,5,128)  Add 5 wide border of val 128.
    imnew = addborder (im,5,5,'even')  Even reflection.
    imnew = addborder (im,5,5,'odd')  Odd reflection.
    imnew = addborder (im,5,5,'wrap')  Wraparound.
    """
    ysize, xsize = im.shape
    
    
#     check thickness
    if (xbdr > xsize-1) or (ybdr > ysize-1):
        raise ValueError('borders must be thinner than image')
    
#     if arg is a number, fill border with its value.
    if isinstance(arg, (int, float)):
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_CONSTANT, value=arg)
    
#     Even reflections
    elif arg == 'even':
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_REFLECT)
        
#     Odd reflections
    elif arg == 'odd':
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_REFLECT_101)
        
#    Wraparound
    elif arg == 'wrap':
        imbig = cv2.copyMakeBorder(im, ybdr,ybdr,xbdr,xbdr, cv2.BORDER_WRAP)
    else:
        raise ValueError('unknown border style')
    return imbig

In [7]:
def filt2(kernel, im1, reflect_style='odd'):
    """
    im2 = filt2(kernel,im1,reflect_style)
    Improved version of filter2 in MATLAB, which includes reflection.
    Default style is 'odd'. Also can be 'even', or 'wrap'.
    im2 = filt2(kern,image)  apply kernel with odd reflection (default).
    im2 = filt2(kern,image,'even')  Use even reflection.
    im2 = filt2(kern,image,128)  Fill with 128's.

    Ruth Rosenholtz
    """
    
    ky,kx = kernel.shape
    iy,ix = im1.shape

    # TODO: index should be checked, maybe it needs to be changed 1 pixel (removing -1)
    imbig = addborder(im1, kx-1, ky-1, reflect_style)
    imbig = conv2(imbig, kernel, 'same')
    im2 = imbig[ky:ky-1+iy, kx:kx-1+ix]


    return im2

In [8]:
def RRcontrast1channel(pyr, DoG_sigma=2):
    
    levels = len(pyr)
    contrast = [0] * levels
    
    innerG1, outerG1 = DoG1filter(round(DoG_sigma*3), DoG_sigma)

    innerG1 = innerG1[:,None]
    outerG1 = outerG1[:,None]

    for i in range(0,levels):
        inner = filt2(innerG1, pyr[(i,0)])
        inner = filt2(innerG1.T, inner)
        outer = filt2(outerG1, pyr[(i,0)])
        outer = filt2(outerG1.T, outer)
        tmp = inner - outer
        contrast[i] = abs(tmp)
 
    return contrast

In [9]:
def contrastClutter(inputImage, numlevels, filt_sigma, pool_sigma=None, pix=1):
    
    if pool_sigma is None:
        pool_sigma = 3 * filt_sigma

    if isinstance(inputImage, list):
        L_pyr = inputImage[0]
        
    else:
        if isinstance(inputImage, str):
            im = cv2.imread(inputImage)
            if im is None:
                print('Unable to open %s image file.') #TODO: add logger
                return 0

        elif isinstance(inputImage,np.ndarray):
            im = inputImage
             
        m, n, d = im.shape
        if d == 3:
            Lab = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
            L = Lab[:,:,0]
        else:
            L = im

  
        pyr = pt.pyramids.GaussianPyramid(L, height=numlevels)
        L_pyr = pyr.pyr_coeffs
        
        # TODO
    contrast = RRcontrast1channel(L_pyr, filt_sigma)
    
    m, n = len(contrast), 1
    clutter_levels = [0] * m
    bigG = RRgaussfilter1D(round(pool_sigma*2), pool_sigma)[:,None]
    
    
    
    for scale in range(0,m):
        for channel in range(0,n):
    #         var(X) = E(X.^2) - E(X).^2
    #         get E(X) by filtering X with a 1-D Gaussian window separably in x and y directions
            meanD = RRoverlapconv(bigG, contrast[scale])
            meanD = RRoverlapconv(bigG.T, meanD)
    #         get E(X.^2) by filtering X.^2 with a 1-D Gaussian window separably in x and y directions
            meanD2 = RRoverlapconv(bigG, contrast[scale] ** 2)
            meanD2 = RRoverlapconv(bigG.T, meanD2)

    #         get variance by var(X) = E(X.^2) - E(X).^2
            stddevD = np.sqrt(abs(meanD2 - meanD ** 2))
            clutter_levels[scale] = stddevD

            
            
        
    kernel_1d = np.array([[0.05, 0.25, 0.4, 0.25, 0.05]])
    kernel_2d = signal.convolve2d(kernel_1d, kernel_1d.T)
    
    clutter_map = clutter_levels[0]
    for scale in range(1,m):
        clutter_here = clutter_levels[scale]
        
        for kk in range(scale, 2, -1):
            # TODO
#             start= np.array([[0,0]])
#             step = np.array([[2,2]])
#             stop = step * (np.floor( (start - np.ones_like(start))/step) + len(clutter_here))
            clutter_here = pt.upConv(image=clutter_here, filt=kernel_2d, edge_type='reflect1', step=[2,2], start=[0,0])
    
        common_sz = min(clutter_map.shape[0], clutter_here.shape[0]), min(clutter_map.shape[1], clutter_here.shape[1])
        for i in range(0, common_sz[0]):
            for j in range(0, common_sz[1]):
                 clutter_map[i][j] = max(clutter_map[i][j], clutter_here[i][j])

    return clutter_levels, clutter_map

## COMPUTECLUTTER

In [10]:
def computeClutter(inputImage, numlevels=3, contrast_filt_sigma=1, contrast_pool_sigma=None, color_pool_sigma=3, contrast_pix=0, color_pix=0, orient_pix=0):
    
    if contrast_pool_sigma is None:
        contrast_pool_sigma = 3 * contrast_filt_sigma

    orient_pool_sigma = 7/2

    if isinstance(inputImage, str):
        im = cv2.imread(inputImage)
        if im is None:
            print('Unable to open %s image file.') #TODO: add logger
            return 0
        else:
            m, n, d = im.shape
            if d == 3:
                Lab = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
                RRLab = [0, 0, 0]
            else:
                print('should be run on RGB color images.  Input image appears to be grayscale.\n')
                return 0
  
    else:
        pass #TODO 

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
    RRLab[0] = pyr.pyr_coeffs

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,1], height=numlevels)
    RRLab[1] = pyr.pyr_coeffs

    pyr = pt.pyramids.GaussianPyramid(Lab[:,:,0], height=numlevels)
    RRLab[2] = pyr.pyr_coeffs
        
    
#     # compute the color clutter
    color_clutter_levels, color_clutter_map = colorClutter(RRLab, numlevels, color_pool_sigma, color_pix)
#     # compute the contrast clutter
    contrast_clutter_levels, contrast_clutter_map = contrastClutter(RRLab, numlevels, contrast_filt_sigma, contrast_pool_sigma, contrast_pix)
#     #compute the orientation clutter
#     orient_clutter_levels, orientation_clutter_map = orientationClutter(RRLab, numlevels, orient_pix)

#     # output them in cell structures
    color_clutter = [color_clutter_levels, color_clutter_map]
    contrast_clutter = [contrast_clutter_levels, contrast_clutter_map]
#     orientation_clutter = [orient_clutter_levels, orientation_clutter_map]


    # return color_clutter, contrast_clutter, orientation_clutter
    return color_clutter, contrast_clutter

In [11]:
# def getClutter_Fc(filename, p=1):
#     color_clutter, contrast_clutter, orient_clutter = computeClutter(filename, 3, 1, 3, 3, 0, 0, 0)
#     clutter_map_fc = color_clutter{2} / 0.2088 + contrast_clutter{2} / 0.0660 + orient_clutter{2} / 0.0269
#     clutter_scalar_fc = mean(clutter_map_fc(:).^ p).^ (1 / p) #element wise
#     return clutter_scalar_fc, clutter_map_fc

# clutter_scalar_fc, clutter_map_fc = getClutter_FC('test.jpg');

In [12]:
# test

color_clutter, contrast_clutter = computeClutter('test.jpg', 3, 1, 3, 3, 0, 0, 0)

img = Image.fromarray(contrast_clutter[1]/0.01) #0.01 for test: more brightness
img.show()